In [1]:
import pandas as pd
import requests
import json
from IPython.display import JSON
from IPython.display import JSON

### We will use the AeroDataBox API, which can retrieve all sorts of information about flights and airports

In [2]:
import requests
import pandas as pd
import sqlalchemy
from api_keys import rapidApi_key as key
from api_keys import mysql_key as sql_key


In [20]:
# check to see if we can get airport information for Berlin 

url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

querystring = {"lat":"52.31",
               "lon":"13.24",
               "radiusKm":"50",
               "limit":"10",
               "withFlightInfoOnly":"true",
              "withleg": True,
              "withLocation": True}

headers = {
	"X-RapidAPI-Key": key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'searchBy': {'lat': 52.31, 'lon': 13.24}, 'count': 1, 'items': [{'icao': 'EDDB', 'iata': 'BER', 'name': 'Berlin Brandenburg', 'shortName': 'Brandenburg', 'municipalityName': 'Berlin', 'location': {'lat': 52.35139, 'lon': 13.493889}, 'countryCode': 'DE'}]}


In [4]:
# view the response as .json instead of .text 
response.json()

{'searchBy': {'lat': 52.31, 'lon': 13.24},
 'count': 1,
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE'}]}

In [5]:
# turn it into a dataframe 
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,52.35139,13.493889


In [6]:
response.json()

{'searchBy': {'lat': 52.31, 'lon': 13.24},
 'count': 1,
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE'}]}

### Get airport information for other cities from the cities table

In [7]:
# call cities_df into this file
cities_df = pd.read_csv('cities.csv')
print(cities_df.info())
cities_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        4 non-null      object 
 1   country     4 non-null      object 
 2   latitude    4 non-null      float64
 3   longitude   4 non-null      float64
 4   population  4 non-null      int64  
 5   city_id     4 non-null      int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 324.0+ bytes
None


,city,country,latitude,longitude,population,city_id
0,Berlin,Germany,52.3112,13.2418,3850809,1
1,London,United Kingdom,51.3026,0.7390,8799800,2
2,Manchester,United Kingdom,53.2846,2.1443,551938,3
3,Istanbul,Turkey,41.0049,28.5718,15907951,4


In [8]:
airports_info_list = []

for i, city in enumerate(cities_df['city_id']):
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    
    querystring = {"lat":cities_df.iloc[i]["latitude"],"lon":cities_df.iloc[i]['longitude'],"radiusKm":"100","limit":"10","withFlightInfoOnly":"true"}
    
    headers = {
        "X-RapidAPI-Key": key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    
    airports_info_list.append(response.json()['items'])

In [10]:
airports_info_list[0]

[{'icao': 'EDDB',
  'iata': 'BER',
  'name': 'Berlin Brandenburg',
  'shortName': 'Brandenburg',
  'municipalityName': 'Berlin',
  'location': {'lat': 52.35139, 'lon': 13.493889},
  'countryCode': 'DE'}]

In [11]:
## only take icao and city_name from response.json()
airports_dict = {
                'icao':[],
                'city_name':[]}


for i in range(len(airports_info_list)):
    for j in range(len(airports_info_list[i])):
        #if item:  # Check if the item is not empty
            airports_dict['icao'].append(airports_info_list[i][j]['icao'])
            airports_dict['city_name'].append(airports_info_list[i][j]['municipalityName'])

In [12]:
airports_df = pd.DataFrame(airports_dict)

In [13]:
airports_df

,icao,city_name
0,EDDB,Berlin
1,EGMC,Southend
2,EGLC,London
3,EGKR,Redhill
4,EGKK,London
5,EGSS,London
6,EGSH,Norwich
7,LTFM,Istanbul
8,LTFJ,Istanbul


In [14]:
# merge airports_df with cities_df 
airports_df = airports_df.merge(cities_df, how='inner', left_on='city_name', right_on='city')[["icao", "city_id"]]

In [15]:
airports_df

,icao,city_id
0,EDDB,1
1,EGLC,2
2,EGKK,2
3,EGSS,2
4,LTFM,4
5,LTFJ,4


In [16]:
#export to csv
airports_df.to_csv('airports.csv', index=False)

## Connect to the MySQL database

In [17]:
airports_df = pd.read_csv('airports.csv')
airports_df

,icao,city_id
0,EDDB,1
1,EGLC,2
2,EGKK,2
3,EGSS,2
4,LTFM,4
5,LTFJ,4


In [21]:
host = '127.0.0.1'
schema = 'gans_local' # The name of your database (schema) might be different
user = 'root'
password = 'sql_key' # YOUR password
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
airports_df.to_sql('airports',con=con,if_exists='append',index=False)

In [23]:
airports_df
print(airports_df)

   icao  city_id
0  EDDB        1
1  EGLC        2
2  EGKK        2
3  EGSS        2
4  LTFM        4
5  LTFJ        4


In [10]:
#!pip install pymysql

In [26]:
#flights_df.to_sql('flights',con=con,if_exists='append',index=False)

1882

## Connect MySQL to AWS

In [47]:
host = 'wbs-project5-db.csv6puy3yaat.eu-north-1.rds.amazonaws.com'
schema = 'gans_aws' # The name of your database (schema) might be different
user = 'admin'
password = 'awskey'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [48]:
airports_df.to_sql('airports',con=con,if_exists='append',index=False)

6

In [49]:
# read it and check it
airports_df = pd.read_sql('airports', con=con)
airports_df

,icao,city_id
0,EDDB,1
1,EGKK,2
2,EGLC,2
3,EGSS,2
4,LTFJ,4
5,LTFM,4
